In [10]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, default_data_collator, \
    DataCollatorForLanguageModeling
from datasets import load_dataset
from arguments import DataTrainingArguments, ModelArguments, TrainingArguments

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base', legacy=True)

# Initialize the data arguments
data_args = DataTrainingArguments(
    datasets='floorplan',
    data_dir=None,
    train_split='train',
    max_seq_length=512,
    max_output_seq_length=512,
    max_seq_length_eval=512,
    max_output_seq_length_eval=512,
    exp='full',
    boundary_in_where='Encoder'
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
model = T5ForConditionalGeneration.from_pretrained('experiments/floorplan-t5-base-full-ep2-len512-b4-train-original-baseline/episode0')

In [16]:
# Example usage
input_sentence = "Balcony 1 is about 8 foot X 2 foot.  This is located right off of the kitchen. Balcony 2 is about 10 foot X 3 foot.  This is facing north of the house. The bathroom is about 8 foot X 6 foot.  It is located between the living room and the kitchen. Common room 1 is about 10 foot X 12 foot.  The common rooms are located diagonal from each other. Common room 2 is about 10 foot X 10 foot.  The common rooms are located diagonal from each other. The kitchen is a 10 foot X 10 foot room.  It looks like it is opened up to the living room. The living room is located on the east side of the house.  It is shaped like a \"T\" and it's the biggest room.  It is approximately 14 feet X 32 feet.  Each room that you come out of you will enter the living room. The master is about 10 feet X 15 feet.  It is located far west of the house.It is a rectangle shape."
input_ids = tokenizer(input_sentence, return_tensors="pt").input_ids

# Generate output
output_ids = model.generate(input_ids, max_new_tokens=512)
output_sentence= tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Output sentence:", output_sentence)

Output sentence: [ master room | x coordinate = 178 | y coordinate = 178 | height = 63 | width = 47 ] [ bathroom 2 | x coordinate = 100 | y coordinate = 143 | height = 45 | width = 18 ] [ common room 1 | x coordinate = 163 | y coordinate = 76 | height = 62 | width = 42 ] [ common room 2 | x coordinate = 100 | y coordinate = 173 | height = 45 | width = 37 ] [ living room | x coordinate = 136 | y coordinate = 120 | height = 116 | width = 97 ] [ bathroom 1 | x coordinate = 142 | y coordinate = 187 | height = 32 | width = 30 ] [ balcony | x coordinate = 204 | y coordinate = 122 | height = 11 | width = 57 ] [ kitchen | x coordinate = 74 | y coordinate = 83 | height = 54 | width = 23 ]


In [2]:
training_args = TrainingArguments()

In [18]:
test_dataset = load_dataset('floorplan', data_args, split='finetune',
                            max_input_length=data_args.max_seq_length,
                            max_output_length=data_args.max_output_seq_length,
                            tokenizer=tokenizer, seed=1)

In [20]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [21]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [22]:
test_dataset.evaluate_dataset(data_args=data_args, model=model, device=device, batch_size=4, output_dir='output')

  0%|          | 0/562 [00:00<?, ?it/s]/Users/thebui/Developer/projects/t2f/venv/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:526: UserWarning: `num_beams` is set to None - defaulting to 1.
  warnings.warn("`num_beams` is set to None - defaulting to 1.", UserWarning)
  2%|▏         | 11/562 [05:14<4:22:34, 28.59s/it]


KeyboardInterrupt: 

In [23]:
data_args

DataTrainingArguments(datasets='floorplan', data_dir=None, eval_datasets=None, train_split='train', max_seq_length=512, max_output_seq_length=512, overwrite_cache=True, train_subset=1, episodes='0', num_beams=None, max_seq_length_eval=512, max_output_seq_length_eval=512, input_format=None, output_format=None, multitask=False, num_shots=None, num_ways=None, num_query=5, chunk_size=128, chunk_overlap=64, chunk_size_eval=None, chunk_overlap_eval=None, eval_nll=False, boundary_in_where='Encoder', exp='full', editing_data=False, output_format_type='original', comment='None')